In [19]:
import pandas as pd
from transformers import AutoTokenizer
import tiktoken
from tqdm import tqdm
import jieba



In [20]:

path1 = r"yentinglin/Taiwan-LLM-7B-v2.0.1-chat"
path2 = r"google/gemma-7b"
path3 = r"INX-TEXT/Bailong-instruct-7B"
path4 = r"MediaTek-Research/Breeze-7B-Instruct-v1_0"
path5 = r"taide/TAIDE-LX-7B-Chat"
path6 = r"taide/Llama3-TAIDE-LX-8B-Chat-Alpha1"

tokenizer_1 = AutoTokenizer.from_pretrained(path1)
tokenizer_2 = AutoTokenizer.from_pretrained(path2)
tokenizer_3 = AutoTokenizer.from_pretrained(path3)
tokenizer_4 = AutoTokenizer.from_pretrained(path4)
tokenizer_5 = AutoTokenizer.from_pretrained(path5)
tokenizer_6 = AutoTokenizer.from_pretrained(path6)
tokenizer_7 = tiktoken.encoding_for_model("gpt-4")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
json_path = r"E:\Tokenizer\Efficiency\Treebanks.json"
df = pd.read_json(json_path, lines=True)

df.head()

,text,gold_tokens
0,看似簡單，只是二選一做決擇，但其實他們代表的是你周遭的親朋好友，試著給你不同的意見，但追根究...,"[看似, 簡單, ，, 只, 是, 二, 選, 一, 做, 決擇, ，, 但, 其實, 他們..."
1,其便當都是買來的，就算加熱也是由媽媽負責（後來揭曉其實是避免帶來厄運），父親則在電視台上班。,"[其, 便當, 都是, 買來, 的, ，, 就算, 加熱, 也是, 由, 媽媽, 負責, （..."
2,這次遊行最大的特色，在於越來越多年輕人上街遊行，而且當中不乏行動激烈的躁少年。,"[這, 次, 遊行, 最大, 的, 特色, ，, 在, 於, 越來越, 多, 年輕, 人, ..."
3,懷孕期為421至457日。,"[懷孕, 期, 為, 421, 至, 457, 日, 。]"
4,婷婷向昏迷中的婆婆訴說，為什麼生活會與她想像的不一樣。,"[婷婷, 向, 昏迷, 中, 的, 婆婆, 訴說, ，, 為, 什麼, 生活, 會, 與, ..."


In [22]:
length = len(df)
gold_list = []
character_list = []
taiwan_list = []
gemma_list = []
bailong_list = []
breeze_list = []
taide_list = []
llama3_list = []
gpt_list = []
jieba_list = []

for i in tqdm(range(length)):
    text = df.iloc[i]["text"]
    gold_length = len(df.iloc[i]["gold_tokens"])
    character_length = len(text)
    taiwan_length = len(tokenizer_1.tokenize(text))
    gemma_length = len(tokenizer_2.tokenize(text))
    bailong_length = len(tokenizer_3.tokenize(text))
    breeze_length = len(tokenizer_4.tokenize(text))
    taide_length = len(tokenizer_5.tokenize(text))
    llama3_length = len(tokenizer_6.tokenize(text))
    gpt_length = len(tokenizer_7.encode(text))
    jieba_length = len(jieba.lcut(text))
    gold_list.append(gold_length)
    character_list.append(character_length)
    taiwan_list.append(taiwan_length)
    gemma_list.append(gemma_length)
    bailong_list.append(bailong_length)
    breeze_list.append(breeze_length)
    taide_list.append(taide_length)
    llama3_list.append(llama3_length)
    gpt_list.append(gpt_length)
    jieba_list.append(jieba_length)

dictt = {
    "gold_length": gold_list,
    "character_length": character_list,
    "taiwan_length": taiwan_list,
    "gemma_length": gemma_list,
    "bailong_length": bailong_list,
    "breeze_length": breeze_list,
    "taide_length": taide_list,
    "llama3_length": llama3_list,
    "gpt4_length": gpt_list,
    "jieba_length": jieba_list
}

new_df = pd.DataFrame(dictt)
new_df.head()

100%|██████████| 4997/4997 [00:03<00:00, 1480.89it/s]


,gold_length,character_length,taiwan_length,gemma_length,bailong_length,breeze_length,taide_length,llama3_length,gpt4_length,jieba_length
0,40,58,99,42,41,42,40,48,83,35
1,29,46,93,31,30,30,31,46,73,29
2,25,39,64,27,21,25,25,34,57,24
3,8,13,22,13,10,13,13,11,14,8
4,19,27,56,19,19,19,20,26,50,19


The efficiency is defined as the number of tokens required to represent Traditional Chinese text with respect to the number of gold standard tokens on the same text, i.e.,

$\huge \text{Efficiency} = \frac{\#\text{token}_{\text{standard}}}{\#\text{token}_{\text{model}}}$

In [23]:
all_gold = new_df["gold_length"].sum()
all_character = new_df["character_length"].sum()
all_taiwan = new_df["taiwan_length"].sum()
all_gemma = new_df["gemma_length"].sum()
all_bailong = new_df["bailong_length"].sum()
all_breeze = new_df["breeze_length"].sum()
all_taide = new_df["taide_length"].sum()
all_llama3 = new_df["llama3_length"].sum()
all_gpt4 = new_df["gpt4_length"].sum()
all_jieba = new_df["jieba_length"].sum()

character_efficiency = round(all_gold/all_character, 3)
taiwan_efficiency = round(all_gold/all_taiwan, 3)
gemma_efficiency = round(all_gold/all_gemma, 3)
bailong_efficiency = round(all_gold/all_bailong, 3)
breeze_efficiency = round(all_gold/all_breeze, 3)
taide_efficiency = round(all_gold/all_taide, 3)
llama3_efficiency = round(all_gold/all_llama3, 3)
gpt_efficiency = round(all_gold/all_gpt4, 3)
jieba_efficiency = round(all_gold/all_jieba, 3)


dictt = {"Character": [character_efficiency], "TAIWAN-LLM": [taiwan_efficiency], "Gemma": [gemma_efficiency], "Bailong": [bailong_efficiency], "Breeze": [breeze_efficiency], "Taide 7B": [taide_efficiency], "Llama 3": [llama3_efficiency], "GPT-4": [gpt_efficiency], "jieba": [jieba_efficiency]}
efficiency_df = pd.DataFrame(dictt)

efficiency_df


,Character,TAIWAN-LLM,Gemma,Bailong,Breeze,Taide 7B,Llama 3,GPT-4,jieba
0,0.63,0.392,0.802,1.005,0.847,0.818,0.696,0.444,1.081
